## Imports

In [125]:
import pandas as pd
from random import uniform
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from scipy import constants
from scipy.optimize import leastsq
from scipy import stats
from lmfit import minimize, Parameters, report_fit
%matplotlib inline
    
plt.rcParams['figure.figsize'] = (20, 10)
pd.options.mode.chained_assignment = None  # default='warn'

## Constants

In [126]:
k = constants.value('Boltzmann constant in eV/K')
e = np.exp(1) 

## Read in data

In [127]:
GRDF = pd.read_csv("../Results/Sorted_data.csv")

In [128]:
def schlfld_vals(id, df):
    
    df = df.reset_index(drop = True)

    vals = {'id'    : id,
            'xVals' : np.asarray(df.UsedTemp[df.NewID == id]),
            'yVals' : np.asarray(df.STVlogged[df.NewID == id]),
            'B0'    : e**(df.B0[df.NewID == id].iloc[0]),
            'E'     : abs(df.E[df.NewID == id].iloc[0]),
            'El'    : abs(df.E[df.NewID == id].iloc[0]),
            'Eh'    : df.Eh[df.NewID == id].iloc[0],
            'Tl'    : df.Tl[df.NewID == id].iloc[0], 
            'Th'    : df.Th[df.NewID == id].iloc[0]}
    
    return vals

In [129]:
scfld_vals(37, GRDF)

{'B0': 5.7809725014421308e-07,
 'E': 1.2287902379117677,
 'Eh': 1.3496560636496351,
 'El': 1.2287902379117677,
 'Th': 300.14999999999998,
 'Tl': 278.14999999999998,
 'id': 37,
 'xVals': array([ 278.15,  281.15,  283.15,  285.15,  288.15,  293.15,  298.15,
         300.15,  301.15,  303.15,  305.15,  308.15]),
 'yVals': array([-15.87660296, -14.55849011, -14.17020334, -13.77245107,
        -13.1918524 , -12.50430399, -12.25566174, -12.25444623,
        -12.30681422, -12.44085777, -12.71036104, -13.66036831])}

In [130]:
def schlfld_residuals(sparams, x, data):
    B0 = sparams['B0'].value
    E  = sparams['E'].value
    El = sparams['El'].value
    Eh = sparams['Eh'].value
    Tl = sparams['Tl'].value
    Th = sparams['Th'].value
    e  = sparams['e'].value
    k  = sparams['k'].value
    
    model = np.log((B0*e**((-E/k)*((1/x)-(1/283.15))))/(
                   1+(e**((El/k)*((1/Tl)-(1/x))))+(e**((Eh/k)*((1/Th)-(1/x))))))
    
    return model - data

In [131]:
def schlfld_model(vals):
    
    xVals    = vals["xVals"]
    ldata    = vals["yVals"]
    
    res = {'id'     : vals["id"],
           'B0'     : vals["B0"],
           'E'      : vals["E"],
           'El'     : vals["El"],
           'Eh'     : vals["Eh"],
           'Tl'     : vals["Tl"],
           'Th'     : vals["Th"],
           'chisqr' : 0,  # will test on each try for improvment
           'aic'    : [np.NaN],
           'bic'    : [np.NaN]}
    
    trycount = 0
    while True:

        trycount += 1
        
        if trycount > 10:
            res = pd.DataFrame(res)
            if res["chisqr"].values[0] == 0:
                print("Failed to converge!")
            else:
                print("Converged! Highest chisqr:", res["chisqr"].values[0])             
            break

        try:
            print(id, 'Try:', trycount)

            sparams = Parameters()
            sparams.add('B0', value = uniform(vals["B0"]*.2, vals["B0"]*1.8), min = 0)
            sparams.add('E',  value = uniform(vals["E"]*.2, vals["E"]*1.8), min= 0)
            sparams.add('El', value = uniform(vals["E"]*.2, vals["E"]*1.8), min = 0)
            sparams.add('Eh', value = uniform(vals["Eh"]*.2, vals["Eh"]*1.8), min = 0)
            sparams.add('Tl', value = vals["Tl"], min = 260, max = 330)
            sparams.add('Th', value = vals["Th"], min = 260, max = 330)
            sparams.add('e',  value = e, vary = False)
            sparams.add('k',  value = k, vary = False)

            out = minimize(schlfld_residuals, sparams, args = (xVals, ldata))
            
            print("chisqr:", out.chisqr)

            if out.chisqr > res["chisqr"]:
                res = {'id'     : [id],
                       'B0'     : [out.params["B0"].value],
                       'E'      : [out.params["E"].value],
                       'El'     : [out.params["El"].value],
                       'Eh'     : [out.params["Eh"].value],
                       'Tl'     : [out.params["Eh"].value],
                       'Th'     : [out.params["Eh"].value],
                       'chisqr' : [out.chisqr],
                       'aic'    : [out.aic],
                       'bic'    : [out.bic]}

            continue

        except ValueError:
            print('Did not converge.')
            continue
    
    return res

In [136]:
schfld_model(scfld_vals(37, GRDF))

3 Try: 1
chisqr: 0.0684082181234
3 Try: 2
Did not converge.
3 Try: 3
chisqr: 0.068408216757
3 Try: 4
chisqr: 0.435134531188
3 Try: 5
chisqr: 0.0684082195423
3 Try: 6
chisqr: 0.0684082198446
3 Try: 7
chisqr: 0.0684082192045
3 Try: 8
chisqr: 0.0684082173257
3 Try: 9
chisqr: 0.0684082184251
3 Try: 10
chisqr: 0.0684082173011
Converged! Highest chisqr: 0.435134531188


,B0,E,Eh,El,Th,Tl,aic,bic,chisqr,id
0,5.974889e-07,1.596683,15.946197,2.968725,15.946197,15.946197,-27.80408,-24.89464,0.435135,3


In [117]:
res["chisqr"].values[0]

0.43513453125257218